# Testing the normal label formulation and the DL Enhancement / Lifted Formulation from https://link.springer.com/content/pdf/10.1007/s13676-012-0010-0.pdf

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
import tsplib95
from imports import networkx as imp_nx
from network import min_tsp, gen_path_atsp, atsp_DT_formulation
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt



def __write_solution(solution:list):
    print("writing solutions...")
    workbook = xlsxwriter.Workbook('BenchmarksTSP.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('F1', 'Objective Value', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark(paths:list):
    solutions = []
    i = 0
    
    for path in paths:
        tsp = tsplib95.load(path)
        
        G = tsp.get_graph()
        G = imp_nx.read(G)
        #m = gen_path_atsp.createGenModel(G,'min','metric')
        m = atsp_DT_formulation.createGenModel(G, "min", "metric")
        m.params.MIPGap = 0.05
        m.params.TimeLimit = 1500
       

        m.optimize()

        solutions.append((i, len(G.G.edges()), len(G.G.nodes()), m.runTime, m.objVal, m.numConstrs, m.MIPGap))
        i += 1
    print(solutions)
    __write_solution(solutions)         

if __name__ == '__main__':
    

    paths = ["TSPTestInstances/kroB150.tsp",
             "TSPTestInstances/bayg29.tsp",
             "TSPTestInstances/bier127.tsp",
             ]
    
    runBenchmark(paths)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-03-23
Using license file C:\Program Files\Gurobi\gurobi.lic
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 1500.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 22801 rows, 22650 columns and 178061 nonzeros
Model fingerprint: 0xd22872de
Variable types: 0 continuous, 22650 integer (22500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [8e+00, 4e+03]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]


KeyboardInterrupt: 

# Heuristic implementation. Using Nearest Neighbour to create a new Tour and 2 Opt to improve the tour

In [2]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
from imports import networkx as imp_nx
from network import tsp_heuristics as tsph
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt


nbr_cities = 150

path = r"TSPTestInstances/kroB150.xml"
distance = readFile.read_tsplib(path)
for i in range(nbr_cities):
    distance[i][i] = 10000 # Changing the diagonal to "infinity"
        
tour, length = tsph.nearNeighHeur(distance, nbr_cities)
tour, length = tsph.twoOptImprov(distance, tour, length, nbr_cities)
print(tour)
print(length)



[0, 52, 136, 6, 81, 8, 77, 25, 60, 23, 116, 31, 131, 141, 124, 45, 11, 39, 57, 128, 101, 22, 29, 67, 34, 1, 103, 106, 129, 64, 146, 79, 76, 114, 139, 59, 115, 3, 13, 47, 86, 95, 149, 51, 10, 83, 127, 102, 65, 43, 62, 50, 126, 145, 134, 15, 119, 36, 28, 17, 48, 92, 142, 104, 18, 140, 98, 91, 9, 135, 143, 96, 80, 44, 32, 99, 35, 56, 73, 68, 117, 72, 2, 58, 40, 88, 121, 20, 138, 130, 75, 120, 110, 69, 90, 122, 93, 94, 49, 132, 85, 42, 4, 118, 55, 100, 125, 113, 107, 112, 137, 61, 82, 71, 123, 70, 37, 38, 27, 87, 147, 97, 108, 144, 33, 89, 24, 16, 105, 7, 111, 21, 74, 133, 53, 148, 5, 46, 30, 66, 41, 54, 19, 63, 109, 78, 12, 14, 26, 84, 0]
26244.22478014847
